<a href="https://colab.research.google.com/github/Nithish4603/nithish/blob/main/SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk

In [21]:
#loading the data
df = pd.read_csv('train-v2.tsv', sep='\t',header= None)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78609 entries, 0 to 78608
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       78609 non-null  int64 
 1   1       78609 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [23]:
df.columns = ['class','tweet']
df


,class,tweet
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?
...,...,...
78604,1,"@USER @USER diolch yn fawr am ein diweddaru, r..."
78605,1,@USER @USER @USER diolch! Digon o opsiynne felly!
78606,0,mor hungover heddiw 🔫
78607,1,"@USER eidiaaaal, awni dydd llun nesa ia? Y air..."


In [24]:
#  removing @USER
df['tweet'] = df['tweet'].str.replace('@USER','')

In [25]:
# removing special characters , puntuations and numbers
df['tweet'] = df['tweet'].str.replace("[^a-zA-Z#]", " ")
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,class,tweet
0,0,a sicrhau bod mwy o arian poced da ti nes ...
1,1,Parti Dolig da gyda tim swyddfa canolog Ty Gw...
2,0,yeaah ma fe yn wir oh well
3,1,hahaha idk am oedd y bws ti
4,0,dwim yn gal llun ohoni


In [26]:
## refered from https://www.youtube.com/watch?v=RLfUyn3HoaE&t=1489s

In [27]:
# individual words are considered as tokens 
tokenized_tweet = df['tweet'].apply(lambda x:x.split())
tokenized_tweet.head()

0    [a, sicrhau, bod, mwy, o, arian, poced, da, ti...
1    [Parti, Dolig, da, gyda, tim, swyddfa, canolog...
2                   [yeaah, ma, fe, yn, wir, oh, well]
3                  [hahaha, idk, am, oedd, y, bws, ti]
4                         [dwim, yn, gal, llun, ohoni]
Name: tweet, dtype: object

In [28]:
#Joining the words
for i in range(len(tokenized_tweet)):
  tokenized_tweet[i] = " ".join(tokenized_tweet[i])

df['tweet'] = tokenized_tweet
df.head()  

,class,tweet
0,0,a sicrhau bod mwy o arian poced da ti nes i fe...
1,1,Parti Dolig da gyda tim swyddfa canolog Ty Gwy...
2,0,yeaah ma fe yn wir oh well
3,1,hahaha idk am oedd y bws ti
4,0,dwim yn gal llun ohoni


In [29]:
stop = ['a' ,'am', 'mewm gwirionedd', 'bron', 'hefyd', 'er', 'bob amser', 'yn', 'an', 'a', 'unrhyw', 'yn', 'fel', 'yn', 'death', 'dod', 'ond', 'gan', 'can', 'gwnaeth', 'gwneud', 'yn gwneud', 'yr un', 'chwaith', 'arall', 'canys', 'rhag', 'wedi', 'cael', 'hyny', 'sut', 'ff', 'mewn', 'yn', 'mae’n', 'ei', 'os', 'yn unig', 'gall', 'efallai', 'mi', 'nerth', 'mwynglawdd', 'rhaid', 'fy', 'nac ychwaith', 'o', 'iawn', 'pryd', 'lle', 'tra', 'lle bynnag', 'pryd bynnag', 'boed', 'sydd', 'tra', 'sefydliad lechyd y byd', 'pwy', 'pwy bynnag', 'y', 'mae ei', 'pam', 'ewyllys', 'gyda', 'fewn', 'heb', 'byddai', 'oes', 'eto', 'ti', 'eich']

In [30]:
#extracting the features
from sklearn.feature_extraction.text import CountVectorizer
bagofwords_vectorizer = CountVectorizer(stop_words=stop)
bagofwords = bagofwords_vectorizer.fit_transform(df['tweet'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['amser', 'bob', 'byd', 'bynnag', 'gwirionedd', 'lechyd', 'mae', 'mewm', 'nac', 'sefydliad', 'un', 'unig', 'ychwaith', 'yr'] not in stop_words.
  % sorted(inconsistent)


In [31]:
bagofwords[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [32]:
# spliting the data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(bagofwords,df['class'], random_state=467, test_size=0.30)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , f1_score

In [34]:
#grid search
from sklearn.model_selection import GridSearchCV
para = {'C' : [0,0.1,0.0001,1,10],"penalty":["l1","l2"]}
grid = GridSearchCV(LogisticRegression(max_iter=1000),para)
grid.fit(x_train,y_train)
grid.best_estimator_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solver

LogisticRegression(C=1, max_iter=1000)

In [39]:
#training the data
model = LogisticRegression(C=0.1,max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(C=0.1, max_iter=1000)

In [40]:
#testing the data
predict = model.predict(x_test)


In [41]:
#f1 score 
f1_score(y_test, predict)

0.7435236445083042

In [38]:
#accuracy score
accuracy_score(y_test,predict)

0.742653606411398